In [32]:
# standard libraries
import pandas as pd
import numpy as np
#import re
import os
from IPython.display import Image
from abc import ABC, abstractmethod
import time
#import sklearn
#import time

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tabulate import tabulate
from IPython.display import clear_output
import xgboost

# data pre-processing
from scipy.io import arff
#from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.impute._base import _BaseImputer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection._split import BaseShuffleSplit
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# prediction models
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.svm._base import BaseSVC 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score

# import warnings filter
import warnings
warnings.filterwarnings('ignore')
from warnings import simplefilter 
simplefilter(action='ignore', category=FutureWarning)

In [5]:
from os import listdir
from os.path import isfile, join

class FilePathManager:
    def __init__(self, local_dir: str):
        self.local_dir = local_dir
    
    def retrieve_full_path(self):
        return os.getcwd()+'/'+self.local_dir

In [6]:
class Loader:
    df = pd.DataFrame()
    
    @abstractmethod
    def load_data(self, file_name):
        pass
    
    @abstractmethod
    def get_df(self):
        pass
    
    def size(self):
        return len(self.df)
        

In [7]:
from typing import Callable

 
class CSVLoader(Loader):
    def __init__(self, file_path_manager: FilePathManager):
        self.file_path_manager = file_path_manager
        
    def load_data(self, _prepare_data: Callable[[pd.DataFrame], pd.DataFrame] = None):
        self.df = pd.read_csv(self.file_path_manager.retrieve_full_path())
        if _prepare_data:
            self.df = _prepare_data(self.df)
    
    def get_df(self):
        return self.df;
    
    def size(self):
        return len(self.df)  

In [58]:
def clean_data(df):
    data = df.copy()
    data.loc[ df['Action']!='allow', 'Action']='not allow'
    depandent = data['Action']
    depandent = (depandent=='allow').astype(int)
    data['Action']= depandent
    return data

In [59]:
loader = CSVLoader(FilePathManager('log2.csv'))
loader.load_data(clean_data)

In [60]:
loader.get_df()['Action'].value_counts()

1    37640
0    27892
Name: Action, dtype: int64

In [61]:
class BaseImputer:
    @abstractmethod
    def fit(self, X, y=None):
        pass
    
    @abstractmethod
    def transform(self, X):
        pass

class BaseModel:
    @abstractmethod
    def fit(self, X, y, sample_weight=None):
        pass
    
    @abstractmethod
    def predict(self, X):
        pass

In [62]:
class Modeling:
    _X_train_fitted = None
    _X_test_fitted = None
    _y_train = None
    _y_test = None
    _y_preds = None
    
    def __init__(self, data: pd.DataFrame, 
                 target_name: str, 
                 shuffle_splitter: BaseShuffleSplit, 
                 imputer: BaseImputer, 
                 model: BaseModel, scaler = None):
        self._data = data
        self._target_name = target_name
        self._shuffle_splitter = shuffle_splitter
        self._imputer = imputer
        self._model = model
        self._X, self._y = self._split_data()
        self._scaler = scaler
        
    @property
    def X(self):
        return self._X
    
    @property
    def y(self):
        return self._y

    @property
    def model(self):
        return self._model
    
    @model.setter
    def model(self, model):
        self._model = model
     
    @property
    def X_train(self):
        return self._X_train_fitted
    
    @property
    def X_test(self):
        return self._X_test_fitted
    
    @property
    def y_train(self):
        return self._y_train
    
    @property
    def y_test(self):
        return self._y_test
    
    @property
    def y_preds(self):
        return self._y_preds
    
    def _split_data(self):
        X = self._data.copy()
        return X.drop([self._target_name], axis=1) , X[self._target_name]
    
    def _shuffle_split(self):
        X = self.X
        y = self.y
        for train_index, test_index in self._shuffle_splitter.split(X,y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]
        return X_train, X_test, y_train, y_test
    
    def _fit_imputer(self, train):
        if self._imputer is not None:
            self._imputer.fit(train)
    
    def _fit_scaler(self, train):
        if self._scaler is not None:
            self._scaler.fit(train)
    
    def _impute_data(self, X: pd.DataFrame):
        if self._imputer is not None:
            return pd.DataFrame(self._imputer.transform(X), columns = self.X.columns, index = X.index)
        return X
    
    def _scale_data(self, X: pd.DataFrame):
        if self._scaler is not None:
            X = pd.DataFrame(self._scaler.transform(X), columns = self._X.columns)
        return X
    
    def prepare(self):
        X_train, X_test, y_train, y_test = self._shuffle_split()   
        self._fit_imputer(X_train)
        X_train = self._impute_data(X_train)
        X_test = self._impute_data(X_test)
        self._fit_scaler(X_train)
        self._X_train_fitted = self._scale_data(X_train)
        self._X_test_fitted = self._scale_data(X_test)
        self._y_train = y_train
        self._y_test = y_test
        
    def prepare_and_train(self):
        self.prepare()
        return self.train()
        
    def train(self):
        self._model.fit(self.X_train, self.y_train)
        self._y_preds = self._model.predict(self.X_train)
        
        return self.metrics(self.y_train, self.y_preds)
        
    def test(self):
        return self.metrics(self.y_test, self._model.predict(self.X_test))
       
    @abstractmethod
    def metrics(self, y_true = None, y_pred = None):
        pass
        

In [63]:
class ClassificationModeling(Modeling):
    def __init__(self, 
                 data: pd.DataFrame, 
                 target_name: str, 
                 shuffle_splitter: BaseShuffleSplit, 
                 imputer: BaseImputer, 
                 model: BaseModel, 
                 scaler = None, 
                 beta: int = 1, 
                 classification: str = 'binary'):
        super().__init__(data, target_name, shuffle_splitter, imputer, model, scaler)
        self.beta = beta
        self.classification = classification
        
    @abstractmethod
    def metrics(self, y_true = None, y_pred = None):
        pass
        

In [165]:
from typing import Type, TypeVar

class SVMClassificationModeling(ClassificationModeling):
    TSVC = TypeVar("TSVC", bound=BaseSVC)
    all_models = [];
    
    def __init__(self, 
             data: pd.DataFrame, 
             target_name: str, 
             shuffle_splitter: BaseShuffleSplit, 
             imputer: BaseImputer, 
             model: BaseModel, 
             scaler = None, 
             beta: int = 1, 
             classification: str = 'binary'):
         super().__init__(data, target_name, shuffle_splitter, imputer, model, scaler, beta, classification)
        
            
    def parameter_tuning(self, params, class_to_instantiate: Type[TSVC]):
        list_of_models = []
        combination = []
        params_base = {}
        output = []
        for key, value in params.items():
            if isinstance(value, list):
                combination.append((key,value))
            else:
                params_base[key]=value
              
        result = SVMClassificationModeling.get_combinations(combination)

        for r in result:
            list_of_models.append(class_to_instantiate(**{**params_base, **r}))
            
        for a_model in list_of_models:
            self.model = a_model
            startTrain = time.time()
            train_metrics = self.train()
            endTrain = time.time()
            test_metrics = self.test()
            endTest = time.time()
            train_time = endTrain - startTrain
            test_time = endTest - endTrain
            output.append({'model': a_model, 'train_metrics': {**train_metrics,**{'elapsed_time':train_time}}, 'test_metrics': {**test_metrics,**{'elapsed_time':test_time}}})
        self.all_models = output
        return output
        
    def find_best_model(self):
        max_accuracy = self.all_models[0]['test_metrics']['accuracy']
        location = 0
        for indx, output_metrics in enumerate(self.all_models):
            if max_accuracy < output_metrics['test_metrics']['accuracy']:
                max_accuracy = output_metrics['test_metrics']['accuracy']
                location = indx
            elif max_accuracy == output_metrics['test_metrics']['accuracy']:
                if output_metrics['test_metrics']['elapsed_time'] < self.all_models[location]['test_metrics']['elapsed_time']:
                    location = indx
                
        return self.all_models[location]
    
    @staticmethod
    def get_combinations(tuples):
        length = len(tuples)
        if length > 1:
            total_params = []
            tuple_copy = tuples.copy()
            a_tuple = tuple_copy.pop(0)
            params_list = get_combinations(tuple_copy)
            for value in a_tuple[1]:
                for a_params in params_list:
                    temp = { a_tuple[0]: value}
                    total_params.append({**temp, **a_params})
            return total_params
        else:
            params_list = []
            a_tuple =  tuples[0]
            for value in a_tuple[1]:
                temp = {}
                temp[a_tuple[0]] = value
                params_list.append(temp)
            return params_list
            
    
    def metrics(self, y_true = None, y_pred = None):
        if y_true is None and y_pred is None:
            y_true = self.y_train
            y_pred = self.y_preds       
        return {'matrix': confusion_matrix(y_true, y_pred), 
                'accuracy': round(accuracy_score(y_true, y_pred), 5), 
                'precision': precision_score(y_true, y_pred, average=self.classification), 
                'recall': recall_score(y_true, y_pred, average=self.classification),
                'f1': f1_score(y_true, y_pred)}

In [166]:
classifier = SVMClassificationModeling(loader.get_df(),'Action',
                           StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=12343),
                           None,
                           None,
                           StandardScaler(), beta=2)

In [167]:
classifier.prepare()

In [253]:
classifier.model = SVC(kernel='linear', C=1)

In [ ]:
## Rerun SVC with different parameters

In [41]:
results = classifier.parameter_tuning({'kernel':['linear','rbf', 'sigmoid'], 'C':[0.1, 0.5, 1, 5, 10, 100, 200, 300, 500, 700], 'degree': 3},SVC)

ValueError: pos_label=1 is not a valid label: array(['allow', 'not allow'], dtype='<U9')

In [424]:
results

[(SVC(C=1, kernel='linear'),
  {'svm_linear': array([1., 1.]),
   '# support vectors': 781,
   'accuracy_train': 0.9978200209277991,
   'accuracy_test': 0.9978200209277991}),
 (SVC(C=2, kernel='linear'),
  {'svm_linear': array([1., 1.]),
   '# support vectors': 590,
   'accuracy_train': 0.9978636205092432,
   'accuracy_test': 0.9978636205092432}),
 (SVC(C=3, kernel='linear'),
  {'svm_linear': array([1., 1.]),
   '# support vectors': 516,
   'accuracy_train': 0.9978636205092432,
   'accuracy_test': 0.9978636205092432}),
 (SVC(C=1),
  {'svm_linear': array([1., 1.]),
   '# support vectors': 1854,
   'accuracy_train': 0.9973186257411929,
   'accuracy_test': 0.9973186257411929}),
 (SVC(C=2),
  {'svm_linear': array([1., 1.]),
   '# support vectors': 1406,
   'accuracy_train': 0.9977546215556331,
   'accuracy_test': 0.9977546215556331}),
 (SVC(C=3),
  {'svm_linear': array([1., 1.]),
   '# support vectors': 1175,
   'accuracy_train': 0.9978200209277991,
   'accuracy_test': 0.9978200209277991})

In [168]:
results = classifier.parameter_tuning({
    "max_iter":1000,
    "loss" : ["hinge", "log", "squared_hinge", "modified_huber", "perceptron"],
    "alpha" : [0.0001, 0.001, 0.01, 0.1],
    "penalty" : ["l2", "l1", "elasticnet", "none"],
},SGDClassifier)

In [157]:
results

[{'model': SGDClassifier(),
  'train_metrics': {'matrix': array([[19507,    17],
          [   85, 26263]]),
   'accuracy': 1.0,
   'precision': 0.9993531202435312,
   'recall': 0.9967739486868074,
   'f1': 0.9980618682070381,
   'elapsed_time': 0.24002885818481445},
  'test_metrics': {'matrix': array([[ 8361,     7],
          [   31, 11261]]),
   'accuracy': 1.0,
   'precision': 0.999378771742989,
   'recall': 0.9972546935883811,
   'f1': 0.9983156028368795,
   'elapsed_time': 0.07587218284606934}},
 {'model': SGDClassifier(penalty='l1'),
  'train_metrics': {'matrix': array([[19487,    37],
          [   14, 26334]]),
   'accuracy': 1.0,
   'precision': 0.9985969436123014,
   'recall': 0.9994686503719448,
   'f1': 0.9990326068400387,
   'elapsed_time': 0.22249865531921387},
  'test_metrics': {'matrix': array([[ 8358,    10],
          [    6, 11286]]),
   'accuracy': 1.0,
   'precision': 0.9991147308781869,
   'recall': 0.9994686503719448,
   'f1': 0.9992916592881176,
   'elapsed_tim

In [169]:
classifier.find_best_model()

{'model': SGDClassifier(loss='perceptron', penalty='none'),
 'train_metrics': {'matrix': array([[19487,    37],
         [   10, 26338]]),
  'accuracy': 0.99898,
  'precision': 0.9985971563981043,
  'recall': 0.9996204645513891,
  'f1': 0.99910854845134,
  'elapsed_time': 0.25165891647338867},
 'test_metrics': {'matrix': array([[ 8358,    10],
         [    4, 11288]]),
  'accuracy': 0.99929,
  'precision': 0.999114887590724,
  'recall': 0.9996457669146298,
  'f1': 0.9993802567507747,
  'elapsed_time': 0.10416007041931152}}

In [3]:
df = pd.read_csv('log2.csv')

In [48]:
port_set = set()
port_dict =  {}
port_columns = df.filter(regex=("Port")).columns
for column in port_columns:
    port_set.update(df[column].unique())
    
for i, port in enumerate(port_set):
    port_dict[port]=i

In [41]:
port_columns

Index(['Source Port', 'Destination Port', 'NAT Source Port',
       'NAT Destination Port'],
      dtype='object')

In [50]:
for column in port_columns:
    df[column+'_encoded'] = df[column].map(port_dict)

In [51]:
df.head()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received,Source Port_encoded,Destination Port_encoded,NAT Source Port_encoded,NAT Destination Port_encoded
0,57222,53,54587,53,allow,177,94,83,2,30,1,1,34082,12,31533,12
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9,33138,1290,33138,1290
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1,2947,27489,22730,27489
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7,27721,1290,27721,1290
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18,27179,29,24306,29


In [68]:
def isGreater(x):
    return x

In [80]:
sal = df['Source Port'].value_counts()


In [98]:
sal[sal > 10].sum()

7132

In [72]:
res

58638     True
27005     True
443       True
57470     True
49418     True
         ...  
63182    False
61133    False
52937    False
1712     False
47111    False
Name: Source Port, Length: 22724, dtype: bool

In [64]:
df['Source Port']

0        57222
1        56258
2         6881
3        50553
4        50002
         ...  
65527    63691
65528    50964
65529    54871
65530    54870
65531    54867
Name: Source Port, Length: 65532, dtype: int64